<img align="center" style="max-width: 1000px" src="banner.png">

<img align="right" style="max-width: 200px; height: auto" src="hsg_logo.png">

##  Lab 05 - Convolutional Neural Networks (CNNs)

GSERM Summer School 2024, Deep Learning: Fundamentals and Applications, University of St. Gallen

The lab environment is based on Jupyter Notebooks (https://jupyter.org), which provide an interactive platform for performing a variety of statistical evaluations and data analyses. In this lab, we will learn how to enhance vanilla **Artificial Neural Networks (ANNs)** using `PyTorch` to classify even more complex images. We will explore a special type of deep neural network known as **Convolutional Neural Networks (CNNs)** to achieve this. CNNs leverage the hierarchical pattern in data, allowing them to assemble more complex patterns from smaller, simpler ones. This hierarchical structure enables CNNs to learn a set of discriminative features and subsequently utilize these learned patterns to classify the content of an image.

The history of CNNs is rich and exhibits pivotal contributions from researchers like *Yann LeCun*, who developed the first practical CNN, known as **LeNet**, in the late 1980s. CNNs have since become a cornerstone in deep learning, significantly advancing the capabilities of image recognition and classification.

In this lab, we will use the `PyTorch` library to implement and train a CNN-based neural network. Our network will be trained on tiny images from the **CIFAR-10** dataset, which includes aeroplanes, cars, birds, cats, deer, dogs, frogs, horses, ships, and trucks. We will utilize the learned CNN model to classify previously unseen images into these distinct categories upon successful training.

The figure below illustrates a high-level view of the machine learning process we aim to establish in this lab.

<img align="center" style="max-width: 900px" src="./splash.png">

(Image of the CNN architecture created via http://alexlenail.me/)

As always, pls. don't hesitate to ask all your questions either during the lab, post them in our CANVAS (StudyNet) forum (https://learning.unisg.ch), or send us an email (using the course email).

Before we start let's watch a motivational video:

In [ ]:
from IPython.display import YouTubeVideo
# The Deep Learning Revolution"
# YouTubeVideo('Dy0hJWltsyE', width=800, height=400)

## 1. Lab Objectives:

After today's lab, you should be able to:

> 1. **Understand Convolutional Neural (CNN) Network Design:** Learn the fundamental concepts and architectural design of CNNs.
> 2. **Implement and Train a CNN Model:** Gain hands-on experience with PyTorch to implement, train, and evaluate CNN models.
> 3. **Apply CNN models for Image Classification:** Use CNNs to classify images from the CIFAR-10 dataset, which includes categories such as aeroplanes, cars, birds, and more.
> 4. **Evaluate and Interpret Model Performance:** Evaluate the CNN model's classification results using accuracy metrics and interpret the confusion matrix.

## 2. Setup of the Jupyter Notebook Environment

Similar to the previous labs, we need to import several Python libraries that facilitate data analysis and visualization. We will primarily use `PyTorch`, `NumPy`, `Scikit-learn`, `Matplotlib`, `Seaborn`, and a few utility libraries throughout this lab:

In [ ]:
# import standard python libraries
import os, urllib, io
from datetime import datetime
import numpy as np

Import `Python` machine learning and deep learning libraries:

In [ ]:
# import the PyTorch deep learning library
import torch, torchvision
import torch.nn.functional as F
from torch import nn, optim
from torch.autograd import Variable

Import `Scikit-learn` classification metrics:

In [ ]:
# import sklearn classification evaluation library
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix

Import `Matplotlib`, `Seaborn` and `PIL` data visualization libraries:

In [ ]:
# import matplotlib, seaborn, and PIL data visualization library
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

Enable inline plotting with `Matplotlib`:

In [ ]:
%matplotlib inline

Create a structure of notebook sub-directories inside of the current **working directory** to store the data and the trained neural network models:

In [ ]:
# create the data sub-directory
data_directory = './data_cifar_10'
if not os.path.exists(data_directory): os.makedirs(data_directory)

# create the models sub-directory
models_directory = './models_cifar_10'
if not os.path.exists(models_directory): os.makedirs(models_directory) 

Set a random `seed` value to obtain reproducible results:

In [ ]:
# init deterministic seed
seed_value = 1234
np.random.seed(seed_value) # set numpy seed
torch.manual_seed(seed_value) # set pytorch seed CPU

Google Colab provides free GPUs for running notebooks. However, if you execute this notebook as is, it will use your device's CPU. To run the lab on a GPU, go to `Runtime` > `Change runtime type` and set the Runtime type to `GPU` in the drop-down menu. Running this lab on a CPU is fine, but you will find that GPU computing is faster. *CUDA* indicates that the lab is being run on a GPU.

Enable GPU computing by setting the device flag and initializing a CUDA seed:

In [ ]:
# set cpu or gpu enabled device
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu').type

# init deterministic GPU seed
torch.mps.manual_seed(seed_value)
torch.cuda.manual_seed(seed_value)

# log type of device enabled
print('[LOG] notebook with {} computation enabled'.format(str(device)))

Let's determine if we have access to a GPU provided by e.g. `Google's Colab` environment:

In [ ]:
!nvidia-smi

## 3. Dataset Download and Data Assessment

The **CIFAR-10 database** (**C**anadian **I**nstitute **F**or **A**dvanced **R**esearch) is a collection of images commonly used to train machine learning and computer vision algorithms. The database is widely used to conduct computer vision research using machine learning and deep learning methods:

<img align="center" style="max-width: 500px; height: 500px" src="./cifar10.png">

(Source: https://www.kaggle.com/c/cifar-10)

Further details on the dataset can be obtained via: *Krizhevsky, A., 2009. "Learning Multiple Layers of Features from Tiny Images",  
( https://www.cs.toronto.edu/~kriz/learning-features-2009-TR.pdf ).*

The CIFAR-10 database contains **60,000 color images** (50,000 training images and 10,000 validation images). The size of each image is 32 by 32 pixels. The collection of images encompasses 10 different classes: airplanes, cars, birds, cats, deer, dogs, frogs, horses, ships, and trucks. Let's define the distinct classes for further analysis:

In [ ]:
cifar10_classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

The dataset contains 6,000 images for each of the ten classes. CIFAR-10 is a straightforward dataset that can teach a computer how to recognize objects in images.

Let's download, transform, and inspect the training images of the dataset. First, we will define the directory where we aim to store the training data:

In [ ]:
train_path = data_directory + '/train_cifar10'

Now, let's download the training data:

In [ ]:
# define pytorch transformation into tensor format
transf = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# download and transform training images
cifar10_train_data = torchvision.datasets.CIFAR10(root=train_path, train=True, transform=transf, download=True)

Verify the volume of training images downloaded:

In [ ]:
# get the length of the training data
len(cifar10_train_data)

Next, let's investigate a couple of the training images:

In [ ]:
# set (random) image id
image_id = 1800

# retrieve image exhibiting the image id
cifar10_train_data[image_id]

Ok, that doesn't seem easily interpretable ;) Let's first separate the image from its label information:

In [ ]:
cifar10_train_image, cifar10_train_label = cifar10_train_data[image_id]

Great, now we can visually inspect our sample image:

In [ ]:
# define tensor to image transformation
trans = torchvision.transforms.ToPILImage()

# set image plot title 
plt.title('Example: {}, Label: "{}"'.format(str(image_id), str(cifar10_classes[cifar10_train_label])))

# un-normalize cifar 10 image sample
cifar10_train_image_plot = cifar10_train_image / 2.0 + 0.5

# plot 10 image sample
plt.imshow(trans(cifar10_train_image_plot))

Fantastic, right? Now, let's decide where we want to store the evaluation data:

In [ ]:
eval_path = data_directory + '/eval_cifar10'

And download the evaluation data:

In [ ]:
# define pytorch transformation into tensor format
transf = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# download and transform validation images
cifar10_eval_data = torchvision.datasets.CIFAR10(root=eval_path, train=False, transform=transf, download=True)

Verify the volume of downloaded validation images:

In [ ]:
# get the length of the training data
len(cifar10_eval_data)

## 4. Neural Network Implementation

In this section we, will implement the architecture of the **neural network** we aim to utilize to learn a model that is capable of classifying the **32x32 pixel** CIFAR 10 images according to the objects contained in each image. However, before we start the implementation, let's briefly revisit the process to be established. The following cartoon provides a birds-eye view:

<img align="center" style="max-width: 900px" src="./process.png">

Our CNN, which we name 'CIFAR10Net', consists of two **convolutional layers** and three **fully-connected layers**. Convolutional layers are generally designed to learn a set of **high-level features** (patterns) in the processed images, such as tiny edges and shapes. The fully-connected layers utilize these learned features to form **non-linear feature combinations** that allow for highly accurate classification of the image content into the different classes of the CIFAR-10 dataset, such as birds, aeroplanes, and horses.

Let's implement the network architecture and subsequently take a more in-depth look at its details:

In [ ]:
# implement the CIFAR10Net network architecture
class CIFAR10Net(nn.Module):
    
    # define the class constructor
    def __init__(self):
        
        # call super class constructor
        super(CIFAR10Net, self).__init__()
        
        # specify convolution layer 1
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1, padding=0, device=device)
        
        # define max-pooling layer 1
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # specify convolution layer 2
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0, device=device)
        
        # define max-pooling layer 2
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # specify fc layer 1 - in 16 * 5 * 5, out 120
        self.linear1 = nn.Linear(16 * 5 * 5, 120, bias=True, device=device) # the linearity W*x+b
        self.relu1 = nn.ReLU(inplace=True) # the non-linearity
        
        # specify fc layer 2 - in 120, out 84
        self.linear2 = nn.Linear(120, 84, bias=True, device=device) # the linearity W*x+b
        self.relu2 = nn.ReLU(inplace=True) # the non-linarity
        
        # specify fc layer 3 - in 84, out 10
        self.linear3 = nn.Linear(84, 10, device=device) # the linearity W*x+b
        
        # add a softmax to the last layer
        self.logsoftmax = nn.LogSoftmax(dim=1) # the softmax
        
    # define network forward pass
    def forward(self, images):
        
        # high-level feature learning via convolutional layers
        
        # define conv layer 1 forward pass
        x = self.pool1(self.relu1(self.conv1(images)))
        
        # define conv layer 2 forward pass
        x = self.pool2(self.relu2(self.conv2(x)))
        
        # feature flattening
        
        # reshape image pixels
        x = x.view(-1, 16 * 5 * 5)
        
        # combination of feature learning via non-linear layers
        
        # define fc layer 1 forward pass
        x = self.relu1(self.linear1(x))
        
        # define fc layer 2 forward pass
        x = self.relu2(self.linear2(x))
        
        # define layer 3 forward pass
        x = self.logsoftmax(self.linear3(x))
        
        # return forward pass result
        return x

You may have noticed that we applied two more layers (compared to the MNIST example described in the last lab) before the fully-connected layers. These layers are called **convolutional layers** and typically consist of three operations: (1) **convolution**, (2) **non-linearity**, and (3) **max-pooling**. These operations are usually executed sequentially during the forward pass through a convolutional layer.

Next, we will take a detailed look at the functionality and number of parameters in each layer. We will start by providing images of 3x32x32 dimensions to the network, i.e., the three channels (red, green, blue) of an image, each of size 32x32 pixels.

### 4.1. High-Level Feature Learning by Convolutional Layers

Let's first look at the convolutional layers of the network, as illustrated in the cartoon above. Convolutional layers generally consist of three sequential operations: (1) convolution, (2) non-linearity, and (3) max-pooling. The following image provides a bird's-eye view of a typical convolutional layer architecture:

<img align="center" style="max-width: 600px" src="./convolutions.png">

**First Convolutional Layer:** The first convolutional layer expects three input channels and will convolve six filters, each of size 3x5x5. Let's briefly revisit how we can perform a convolutional operation on a given image. We need to define a kernel, which is a matrix of size 5x5, for example. To perform the convolution operation, we slide the kernel along the image horizontally and vertically and obtain the dot product of the kernel and the pixel values of the image inside the kernel ('receptive field' of the kernel).

The following illustration shows an example of a discrete convolution:

<img align="center" style="max-width: 800px" src="./convsample.png">

The left grid is called the input (an image or feature map). The middle grid, referred to as the kernel, slides across the input feature map (or image). At each location, the product between each element of the kernel and the input element it overlaps is computed, and the results are summed up to obtain the output at the current location. In general, a discrete convolution is mathematically expressed by:

<center> $y(m, n) = x(m, n) * h(m, n) = \sum^{m}_{j=0} \sum^{n}_{i=0} x(i, j) * h(m-i, n-j)$, </center>

where $x$ denotes the input image or feature map, $h$ the applied kernel, and $y$ the output.

When performing the convolution operation, the 'stride' defines the number of pixels to pass at a time when sliding the kernel over the input. 'Padding' adds pixels to the input image (or feature map) to ensure that the output has the same shape as the input. Let's look at another animated example:

<img align="center" style="max-width: 800px" src="./convsample_animated.gif">

(Source: https://towardsdatascience.com/a-comprehensive-guide-to-convolutional-neural-networks-the-eli5-way-3bd2b1164a53)

The first convolutional layer expects an image of 3x32x32 (3 input channels, each of size 32x32). It convolves 6 filters, each of size 3x5x5, resulting in 6 feature maps of size 28x28. Let’s calculate the size of the feature map obtained by this layer. 

In our implementation, padding is set to 0, and stride is set to 1. As a result, the output size of the convolutional layer becomes 6x28x28, since (32 input pixel - 5 kernel pixel) + 1 = 28. This layer exhibits ((5 kernel pixel x 5 kernel pixel x 3 input channels) + 1) x 6 output channels = 456 parameters.

**First Max-Pooling Layer:** The max-pooling process is a sample-based discretization operation. The objective is to down-sample an input representation (image, hidden-layer output matrix, etc.), reducing its dimensionality and allowing for assumptions to be made about features contained in the sub-regions binned. To conduct such an operation, we again need to define a kernel. Max-pooling kernels are usually a tiny matrix of, e.g, of size 2x2. To perform the max-pooling operation, we slide the kernel along the image horizontally and vertically (similarly to a convolution) and compute the maximum pixel value of the image (or feature map) inside the kernel (the receptive field of the kernel).

The following illustration shows an example of a max-pooling operation:

<img align="center" style="max-width: 500px" src="./poolsample.png">

The left grid is called the input (an image or feature map). The middle grid, referred to as the kernel, slides across the input feature map (or image). We use a stride of 2, meaning the step distance for stepping over our input will be 2 pixels and won't overlap regions. At each location, the max value of the region that overlaps with the elements of the kernel and the input elements it overlaps is computed, and the results are obtained in the output at the current location.

In our implementation, we do max-pooling with a 2x2 kernel and stride 2, which effectively reduces the original image size from 6x28x28 to 6x14x14. Let's look at an exemplary visualization of 64 features learned in the first convolutional layer on the CIFAR-10 dataset:

<img align="center" style="max-width: 700px" src="./cnnfeatures.png">

(Source: Yu, Dingjun, Hanli Wang, Peiqiu Chen, and Zhihua Wei, **"Mixed Pooling for Convolutional Neural Networks"**, International Conference on Rough Sets and Knowledge Technology, pp. 364-375. Springer, Cham, 2014)

**Second Convolutional Layer:** The second convolutional layer expects 6 input channels and will convolve 16 filters, each of size 6x5x5. Since padding is set to 0 and stride is set to 1, the output size is 16x10x10, calculated as (16 input pixels - 5 kernel pixels) + 1 = 10. This layer, therefore, has ((5 kernel pixels x 5 kernel pixels x 6 input channels) + 1 x 16 output channels) = 2,416 parameters.

**Second Max-Pooling Layer:** The second down-sampling layer uses max-pooling with a 2x2 kernel and stride set to 2. This effectively reduces the size from 16x10x10 to 16x5x5.

### 4.2. Flattening of Learned Features

The output of the final max-pooling layer needs to be flattened so that we can connect it to a fully connected layer. This is achieved using the `torch.Tensor.view` method. Setting the parameter of the method to `-1` will automatically infer the number of rows required to handle the mini-batch size of the data.

### 4.3. Learning of Feature Classification

Let's now look at the non-linear layers of the network illustrated below:

<img align="center" style="max-width: 600px" src="./fullyconnected.png">

The first fully connected layer uses 'Rectified Linear Units' (ReLU) activation functions to learn potential nonlinear combinations of features. The layers are implemented similarly to the fifth lab. Therefore, we will only focus on the number of parameters in each fully-connected layer:

**First Fully-Connected Layer:** The first fully-connected layer consists of 120 neurons, thus in total exhibits ((16 input channels x 5 input pixels x 5 input pixels) + 1 bias) x 120 input neurons = 48,120 parameters.

**Second Fully-Connected Layer:** The output of the first fully-connected layer is then transferred to the second fully-connected layer. The layer consists of 84 neurons equipped with ReLU activation functions, and in total exhibits (120 input dimensions + 1 bias) x 84 input neurons = 10,164 parameters.

The output of the second fully-connected layer is then transferred to the output layer (third fully-connected layer). The output layer is equipped with a softmax (which you learned about in the previous lab) and comprises ten neurons, one for each object class contained in the CIFAR-10 dataset. This layer exhibits (84 + 1) x 10 = 850 parameter.

As a result our CIFAR-10 convolutional neural exhibits a total of 456 + 2,416 + 48,120 + 10,164 + 850 = 62,006 parameter.

(Source: https://www.stefanfiott.com/machine-learning/cifar-10-classifier-using-cnn-in-pytorch/)

Now that we have implemented our first Convolutional Neural Network, we are ready to instantiate a network model to be trained:

In [ ]:
model = CIFAR10Net()

Let's push the initialized `CIFAR10Net` model to the enabled computing `device`:

In [ ]:
model = model.to(device)

Let's double-check if our model was deployed to the GPU, if available:

In [ ]:
!nvidia-smi

Once the model is initialized, we can visualize the model structure and review the implemented network architecture before training:

In [ ]:
# print the initialized architectures
print('[LOG] CIFAR10Net architecture:\n\n{}\n'.format(model))

Looks like intended? Brilliant! Finally, let's have a look into the number of model parameters that we aim to train in the next steps of the notebook:

In [ ]:
# init the number of model parameters
num_params = 0

# iterate over the distinct parameters
for param in model.parameters():

    # collect number of parameters
    num_params += param.numel()
    
# print the number of model paramters
print('[LOG] Number of to be trained CIFAR10Net model parameters: {}.'.format(num_params))

Ok, our "simple" CIFAR10Net model already encompasses an impressive number **62,006 model parameters** to be trained.

We are ready to train the network now that we have successfully implemented and defined the three CNN building blocks. However, we must define an appropriate loss function before starting the training. Remember, we aim to train our model to learn a set of model parameters $\theta$ that minimize the classification error of the true class $c^{i}$ of a given CIFAR-10 image $x^{i}$ and its predicted class $\hat{c}^{i} = f_\theta(x^{i})$ as faithfully as possible.

We use the **'Negative Log-Likelihood (NLL)'** loss in this lab. During training, the NLL loss will penalize models that result in a high classification error between the predicted class labels $\hat{c}^{i}$ and their respective true class label $c^{i}$.


Let's instantiate the NLL loss via the following PyTorch command:

In [ ]:
# define the optimization criterion / loss function
nll_loss = nn.NLLLoss()

Let's also push the initialized `nll_loss` computation to the enabled computing `device`:

In [ ]:
nll_loss = nll_loss.to(device)

Based on the loss magnitude of a certain mini-batch, PyTorch automatically computes the gradients. Furthermore, based on the gradient, the library also helps us optimise and update the network parameters $\theta$.

We will use the **Stochastic Gradient Descent (SGD) optimization** and set the `learning rate` to 0.001. With each mini-batch step, the optimizer will update the model parameters $\theta$ according to the degree of classification error (the NLL loss).

In [ ]:
# define learning rate and optimization strategy
learning_rate = 0.001
optimizer = optim.SGD(params=model.parameters(), lr=learning_rate)

Now that we have successfully implemented and defined the three CNN building blocks, let's take some time to review the `CIFAR10Net` model definition as well as the `loss`. Please read the above code and comments carefully and don't hesitate to ask any questions you might have.

## 5. Neural Network Model Training

In this section, we will train our neural network model (as implemented in the section above) using the transformed images. Specifically, we will examine the distinct training steps and how to monitor the training progress.

### 5.1. Preparing the Network Training

So far, we have pre-processed the dataset, implemented the CNN, and defined the classification error. Let's now start to train a corresponding model for **20 epochs** with a **mini-batch size of 128** CIFAR-10 images per batch. This implies that the whole dataset will be fed to the CNN 20 times in chunks of 128 images, yielding **391 mini-batches** (50,000 training images / 128 images per mini-batch) per epoch. After processing each mini-batch, the parameters of the network will be updated.

In [ ]:
# specify the training parameters
num_epochs = 20 # number of training epochs
mini_batch_size = 128 # size of the mini-batches

Furthermore, let's specify and instantiate a corresponding PyTorch data loader that feeds the image tensors to our neural network:

In [ ]:
cifar10_train_dataloader = torch.utils.data.DataLoader(cifar10_train_data, batch_size=mini_batch_size, shuffle=True)

### 5.2. Running the Network Training

Finally, we start training the model. The training procedure for each mini-batch is performed as follows:

>1. Do a forward pass through the CIFAR10Net network, 
>2. Compute the negative log-likelihood classification error $\mathcal{L}^{NLL}_{\theta}(c^{i};\hat{c}^{i})$, 
>3. Do a backward pass through the CIFAR10Net network, and 
>4. Update the parameters of the network $f_\theta(\cdot)$.

We will monitor whether the loss decreases as training progresses to ensure learning while training our CNN model. Therefore, we obtain and evaluate the classification performance of the entire training dataset after each training epoch. Based on this evaluation, we can conclude the training progress and whether the loss is converging (indicating that the model might not improve further).

The following elements of the network training code below should be given particular attention:

>- `loss.backward()` computes the gradients based on the magnitude of the reconstruction loss,
>- `optimizer.step()` updates the network parameters based on the gradient.

In [ ]:
# init collection of training epoch losses
train_epoch_losses = []

# set the model in training mode
model.train()

# train the CIFAR10 model
for epoch in range(num_epochs):
    
    # init collection of mini-batch losses
    train_mini_batch_losses = []
    
    # iterate over all-mini batches
    for i, (images, labels) in enumerate(cifar10_train_dataloader):
        
        # push mini-batch data to computation device
        images = images.to(device)
        labels = labels.to(device)

        # run forward pass through the network
        output = model(images)
        
        # reset graph gradients
        model.zero_grad()
        
        # determine classification loss
        loss = nll_loss(output, labels)
        
        # run backward pass
        loss.backward()
        
        # update network paramaters
        optimizer.step()
        
        # collect mini-batch reconstruction loss
        train_mini_batch_losses.append(loss.data.item())

    # determine mean min-batch loss of epoch
    train_epoch_loss = np.mean(train_mini_batch_losses)
    
    # print epoch loss
    now = datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
    print('[LOG {}] epoch: {} train-loss: {}'.format(str(now), str(epoch), str(train_epoch_loss)))
    
    # set filename of actual model
    model_name = 'cifar10_model_epoch_{}.pth'.format(str(epoch))

    # save current model to GDrive models directory
    torch.save(model.state_dict(), os.path.join(models_directory, model_name))
    
    # determine mean min-batch loss of epoch
    train_epoch_losses.append(train_epoch_loss)

Upon successful training, let's visualize and inspect the training loss per epoch:

In [ ]:
# prepare plot
fig = plt.figure()
ax = fig.add_subplot(111)

# add grid
ax.grid(linestyle='dotted')

# plot the training epochs vs. the epochs' classification error
ax.plot(np.array(range(1, len(train_epoch_losses)+1)), train_epoch_losses, label='epoch loss (blue)')

# add axis legends
ax.set_xlabel("[training epoch $e_i$]", fontsize=10)
ax.set_ylabel("[Classification Error $\mathcal{L}^{NLL}$]", fontsize=10)

# set plot legend
plt.legend(loc="upper right", numpoints=1, fancybox=True)

# add plot title
plt.title('Training Epochs $e_i$ vs. Classification Error $L^{NLL}$', fontsize=10);

Okay, fantastic. The training error converges nicely. We could definitely train the network for a few more epochs until the error converges. But let's stay with the 20 training epochs for now and continue with evaluating our trained model.

## 6. Neural Network Model Evaluation

Before evaluating our model, let's load the best-performing model. Remember that we stored a snapshot of the model after each training epoch to our local model directory. We will now load the last snapshot saved.

In [ ]:
# restore pre-trained model snapshot
best_model_name = 'https://raw.githubusercontent.com/HSG-AIML-Teaching/GSERM2024-Lab/master/lab_05/models/cifar10_model_epoch_19.pth'

# read stored model from the remote location
model_bytes = urllib.request.urlopen(best_model_name)

# load model tensor from io.BytesIO object
model_buffer = io.BytesIO(model_bytes.read())

# init pre-trained model class
best_model = CIFAR10Net()

# load pre-trained models
best_model.load_state_dict(torch.load(model_buffer, map_location=torch.device(device)))

Let's check if the model was loaded successfully:

In [ ]:
# set model in evaluation mode
best_model.eval()

To evaluate our trained model, we need to feed the CIFAR-10 images reserved for evaluation (the images that we didn't use as part of the training process) through the model. Therefore, let's define a corresponding PyTorch data loader that feeds the image tensors to our neural network:

In [ ]:
cifar10_eval_dataloader = torch.utils.data.DataLoader(cifar10_eval_data, batch_size=10000, shuffle=False)

We will now evaluate the trained model using the same mini-batch approach as we did when training the network and derive the mean negative log-likelihood loss of all mini-batches processed in an epoch:

In [ ]:
# init collection of mini-batch losses
eval_mini_batch_losses = []

# iterate over all-mini batches
for i, (images, labels) in enumerate(cifar10_eval_dataloader):

    # push mini-batch data to computation device
    images = images.to(device)
    labels = labels.to(device)
    
    # run forward pass through the network
    output = best_model(images)

    # determine classification loss
    loss = nll_loss(output, labels)

    # collect mini-batch reconstruction loss
    eval_mini_batch_losses.append(loss.data.item())

# determine mean min-batch loss of epoch
eval_loss = np.mean(eval_mini_batch_losses)

# print epoch loss
now = datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
print('[LOG {}] eval-loss: {}'.format(str(now), str(eval_loss)))

Great! The evaluation loss looks in line with our training loss. Let's now inspect a few sample predictions to get an impression of the model quality. We will again pick a random image from our evaluation dataset and retrieve its PyTorch tensor as well as the corresponding label:

In [ ]:
# set (random) image id
image_id = 777

# retrieve image exhibiting the image id
cifar10_eval_image, cifar10_eval_label = cifar10_eval_data[image_id]

Let's now check the true class of the image we selected:

In [ ]:
cifar10_classes[cifar10_eval_label]

Okay, the randomly selected image should contain a two (2). Let's inspect the image accordingly:

In [ ]:
# define tensor to image transformation
trans = torchvision.transforms.ToPILImage()

# set image plot title 
plt.title('Example: {}, Label: {}'.format(str(image_id), str(cifar10_classes[cifar10_eval_label])))

# un-normalize cifar 10 image sample
cifar10_eval_image_plot = cifar10_eval_image / 2.0 + 0.5

# plot cifar 10 image sample
plt.imshow(trans(cifar10_eval_image_plot))

Now, let's compare the true label with the prediction of our model:


In [ ]:
best_model(cifar10_eval_image.unsqueeze(0).to(device))

We can even determine the likelihood of the most probable class:

In [ ]:
cifar10_classes[torch.argmax(best_model(Variable(cifar10_eval_image.unsqueeze(0)).to(device)), dim=1).item()]

Let's now obtain the predictions for all the CIFAR-10 images in the evaluation data:

In [ ]:
predictions = torch.argmax(best_model(next(iter(cifar10_eval_dataloader))[0].to(device)), dim=1)

Next, let's obtain the overall classification accuracy:

In [ ]:
metrics.accuracy_score(cifar10_eval_data.targets, predictions.to('cpu').detach())

Finally, let's also inspect the confusion matrix of the model predictions to determine the major sources of misclassification:

In [ ]:
# determine classification matrix of the predicted and target classes
mat = confusion_matrix(cifar10_eval_data.targets, predictions.to('cpu').detach())

# initialize the plot and define size
plt.figure(figsize=(8, 8))

# plot corresponding confusion matrix
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False, cmap='YlOrRd_r', xticklabels=cifar10_classes, yticklabels=cifar10_classes)
plt.tick_params(axis='both', which='major', labelsize=8, labelbottom = False, bottom=False, top = False, left = False, labeltop=True)

# set plot title
plt.title('CIFAR-10 classification matrix')

# set plot axis lables
plt.xlabel('[true label]')
plt.ylabel('[predicted label]');

Okay, we can easily see that our current model often confuses images of cats and dogs, as well as images of trucks and cars. This is not surprising since those image categories exhibit a high semantic and visual similarity.

## 7. Lab Summary:

In this lab, you successfully accomplished the following key learnings:

> 1. **Understanding Convolutional Neural Network Design:** Mastered the fundamental concepts and architectural design of convolutional neural networks (CNNs), enhancing your comprehension of deep learning models tailored for image classification.
> 2. **Model Implementation and Training:** Developed practical skills in implementing and training a CNN model using PyTorch, applying it to the CIFAR-10 dataset to predict class labels for categories such as aeroplanes, cars, birds, and more.
> 3. **Evaluating Model Performance:** Gained expertise in evaluating the performance of CNN models through metrics such as loss and accuracy, and through the construction and analysis of confusion matrices.

This lab provided insights into designing, implementing, training, and evaluating CNNs for image classification. It equipped you with essential tools and techniques for effective model building, evaluation, and application. These skills are invaluable for succeeding in deep learning.
